In [1]:
import pandas as pd
import sys
import numpy as np
import json
from utils import fen_to_vector


In [2]:
def extract_cp(eval):
    first_pv = eval[0]["pvs"]
    try:
        return first_pv[0]["cp"]
    except KeyError:
        return first_pv[0]["mate"] #TODO: Make real big number pos or neg

    
def extract_depth(eval):
    depth = eval[0]["depth"]
    return depth

def extract_fen(eval):
    return eval

In [8]:
# Fixed this function, uncomment df_depth if we need depth values, otherwise don't
def json_to_dataframe(path, num_chunks, chunksize=1000):
  reader = pd.read_json(path, lines=True, chunksize=chunksize)
  df_fen = pd.DataFrame(columns=["fen"])
  df_cp = pd.DataFrame()
  #df_depth = pd.DataFrame()
  
  i = 0
  for chunk in reader:
      # concatenate all the chunks together into one column
      df_fen = pd.concat([df_fen, chunk["fen"]], axis=0)
      df_cp = pd.concat([df_cp, chunk["evals"].apply(extract_cp)], axis=0)
      #df_depth = pd.concat([df_depth, chunk["evals"].apply(extract_depth)], axis=0)
      i+=1
      print(f"Loading data into dataframe {(i/num_chunks * 100):.2f}% done.")
      sys.stdout.flush()
      
      if i == num_chunks:
          break
          
  df_cp = df_cp.rename(columns={"evals":"cp"})
  #df_depth = df_depth.rename(columns={"evals":"depth"})
  df = pd.concat([df_fen, df_cp], axis=1)
  return df

# get 2500000 data examples
num_chunks = 250
chunksize = 10000
df = json_to_dataframe("lichess_db_eval.jsonl", num_chunks, chunksize)


Loading data into dataframe 0.40% done.
Loading data into dataframe 0.80% done.
Loading data into dataframe 1.20% done.
Loading data into dataframe 1.60% done.
Loading data into dataframe 2.00% done.
Loading data into dataframe 2.40% done.
Loading data into dataframe 2.80% done.
Loading data into dataframe 3.20% done.
Loading data into dataframe 3.60% done.
Loading data into dataframe 4.00% done.
Loading data into dataframe 4.40% done.
Loading data into dataframe 4.80% done.
Loading data into dataframe 5.20% done.
Loading data into dataframe 5.60% done.
Loading data into dataframe 6.00% done.
Loading data into dataframe 6.40% done.
Loading data into dataframe 6.80% done.
Loading data into dataframe 7.20% done.
Loading data into dataframe 7.60% done.
Loading data into dataframe 8.00% done.
Loading data into dataframe 8.40% done.
Loading data into dataframe 8.80% done.
Loading data into dataframe 9.20% done.
Loading data into dataframe 9.60% done.
Loading data into dataframe 10.00% done.

In [9]:
import os
from sklearn.model_selection import train_test_split
train, test_val = train_test_split(df, test_size=0.4)
test, val = train_test_split(test_val, test_size=0.5)

# make data directory
if not os.path.exists("data"):
    os.makedirs("data")
   
# split data into train, test and val 
train.to_csv(f"data\data_train.csv", encoding='utf-8', index=False)
test.to_csv(f"data\data_test.csv", encoding='utf-8', index=False)
val.to_csv(f"data\data_val.csv", encoding='utf-8', index=False)